# Validação das APIs

In [ ]:
from src.api import Client
api = Client('dev')

> Inicializando API Client em=dev
> Utilizando o token=token_dummy_001


In [ ]:
print("> Verificando criptos disponíveis")
print(api.available_cripto())

> Verificando criptos disponíveis
BTC


In [ ]:
print("> Obtendo status da carteira")
print(api.status())

> Obtendo status da carteira
    ticker      quantity
0      BTC  2.000000e-01
1  BTCUSDT  7.437300e-02
2    money  9.998150e+06
3    total  1.000330e+07


In [ ]:
print("> Verificando histórico de todas as movimentações")
print(api.history())

> Verificando histórico de todas as movimentações
   ticker            datetime  side  quantity    price
0     BTC 2023-08-18 20:35:00   buy       0.2  29909.6
1     BTC 2023-08-18 20:35:13  sell       0.2  29920.7
2     BTC 2023-08-19 00:12:07   buy       1.0  29776.0
3     BTC 2023-08-19 00:12:11   buy       0.2  29776.0
4     BTC 2023-08-19 00:14:27  sell       1.4  29770.0
..    ...                 ...   ...       ...      ...
77    BTC 2023-09-16 06:26:41   buy       0.2  25740.9
78    BTC 2023-09-16 06:26:45  sell       0.2  25740.9
79    BTC 2023-09-16 06:27:50   buy       0.2  25751.8
80    BTC 2023-09-16 06:28:19   buy       0.2  25762.3
81    BTC 2023-09-16 06:28:24  sell       0.2  25762.3

[82 rows x 5 columns]


In [ ]:
print("> Obtendo cotação dos últimos 500 minutos (OHLC) de uma cripto")
print(api.cripto_quotation())

> Obtendo cotação dos últimos 500 minutos (OHLC) de uma cripto
   symbol            datetime      open      high       low     close  \
0     BTC 2023-09-16 06:10:00  25743.58  25746.74  25743.57  25746.74   
1     BTC 2023-09-16 06:11:00  25746.73  25746.74  25741.62  25741.62   
2     BTC 2023-09-16 06:12:00  25741.63  25741.63  25741.16  25741.17   
3     BTC 2023-09-16 06:13:00  25741.17  25741.17  25741.16  25741.17   
4     BTC 2023-09-16 06:14:00  25741.16  25744.00  25741.16  25744.00   
5     BTC 2023-09-16 06:15:00  25744.00  25744.00  25743.99  25744.00   
6     BTC 2023-09-16 06:16:00  25744.00  25744.00  25742.20  25742.21   
7     BTC 2023-09-16 06:17:00  25742.21  25742.21  25742.20  25742.21   
8     BTC 2023-09-16 06:18:00  25742.21  25742.21  25742.20  25742.20   
9     BTC 2023-09-16 06:19:00  25742.21  25746.56  25742.20  25746.55   
10    BTC 2023-09-16 06:20:00  25746.55  25747.17  25744.00  25747.16   
11    BTC 2023-09-16 06:21:00  25747.16  25755.59  25747.16  

In [ ]:
print("> Realizando COMPRA de cripto")
print(api.buy(0.2))

> Realizando COMPRA de cripto
Compra realizada com sucesso: 0.2 BTC por 25767.47 a unidade (total de 5153.494000000001) @ 2023-09-16 06:39:22


In [ ]:
print("> Realizando VENDA cripto")
print(api.sell(0.2))

> Realizando VENDA cripto
Venda realizada com sucesso: 0.2 BTC por 25767.47 a unidade (total de 5153.494000000001) @ 2023-09-16 06:39:26


# Train

In [ ]:
!pip install statsmodels
!pip install pyarrow

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
from src.simple_robot import feature_eng
# Baixando os dados de BTC
df = pd.read_parquet('data/BTC-USDT.parquet')

#### Calculando o target (y)

# Calculando qual a média de close dos próximos 10min
df['forward_average'] = df[::-1]['close'].rolling(10).mean()[::-1].shift(-1)

# Target será a diferença percentual do 'forward_average' com o 'close' atual
df['target'] = 100*(df['forward_average'] - df['close']) / df['close']

df.head(3)
# Outra possibilidade: target como a diferença entre o proximo minuto e o atual: df['diff']= -df['close'].diff(-1)

#### Calculando as features (x)
# Toda a parte de criação de features está no arquivo simple_robot.py. Aqui apenas chamamos a função. Isso é útil, pois conseguimos usar a mesma função no momento de colocar o robô em produção
df = feature_eng(df)
df.head(5)

#### Separando em treino/ teste
"""
Separando usando data. Isso é importante, pois precisamos entender se os modelos criados
em um tempo passado continua sendo útil em um tempo futuro.
"""
test_treshold = '2021-06-01 00:00:00'

train = df[df.index <= test_treshold]
test = df[df.index > test_treshold]

X_train = train.drop(columns=['target'])
y_train = train['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

# Modelo linear simples
model = sm.OLS(y_train,X_train).fit()
print(model.summary())
print('\n\n')

#### Resultado do Modelo Linear
y_hat = model.predict(X_test)
MSE = ((y_hat - y_test)**2).mean()
print(f'MSE={MSE}')

MAE = ((y_hat - y_test).abs()).mean()
print(f'MAE={MAE}')

#### Referência
"""
É sempre recomendado ter valores de referência, para saber se seu modelo é ou não melhor do que outras alternativas "naive"
Abaixo, um exemplo de resultado Naive, assumindo todos 0
"""
MSE_assuming_all_zero = (y_test**2).mean()
print(f'MSE_assuming_all_zero={MSE_assuming_all_zero}')

MAE_assuming_all_zero = (y_test.abs()).mean()
print(f'MAE_assuming_all_zero={MAE_assuming_all_zero}')

# Salvando o modelo em um arquivo pickle para ser utilizado nas etapas seguintes
filename = 'data/model_dummy.pickle'
pickle.dump(model, open(filename, 'wb'))

print(f'\nExport realizado para: {filename}')

                                 OLS Regression Results                                
Dep. Variable:                 target   R-squared (uncentered):                   0.006
Model:                            OLS   Adj. R-squared (uncentered):              0.006
Method:                 Least Squares   F-statistic:                              1371.
Date:                Sat, 16 Sep 2023   Prob (F-statistic):                        0.00
Time:                        06:39:51   Log-Likelihood:                     -1.6275e+05
No. Observations:             1984586   AIC:                                  3.255e+05
Df Residuals:                 1984577   BIC:                                  3.256e+05
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

# Robo

In [ ]:
from src.api import Client
from src.simple_robot import my_robot

api = Client('dev')
minutos = 2
my_robot(api, minutos)

> Inicializando API Client em=dev
> Utilizando o token=token_dummy_001

>>>> My Robot @ 2023-09-16 06:40:59.291685
Tendência neutra de -0.008764677376090418. Nenhuma ação realizada
    ticker      quantity
0      BTC  2.000000e-01
1  BTCUSDT  7.437300e-02
2    money  9.998150e+06
3    total  1.000330e+07
>>>> Aguardando 1 min

>>>> My Robot @ 2023-09-16 06:42:08.389733
Tendência neutra de -0.011065740706193056. Nenhuma ação realizada
    ticker      quantity
0      BTC  2.000000e-01
1  BTCUSDT  7.437300e-02
2    money  9.998150e+06
3    total  1.000330e+07
>>>> Aguardando 1 min
